#TODO

1. Extract TIDs from OE_LABELS.csv
2. Use Indiankanoon API to download the judgements
3. Save the html files
4. Run BeautifulSoup to create json file
5. Create index on elasticsearch
6. Run the search code
7. Merge this file with new judgements

In [1]:
#Download the files from Indiankanoon

In [43]:
import pandas as pd
import numpy as np
import re
import html2text
import requests
from urllib.parse import urljoin
from datetime import datetime
import pandas as pd

In [44]:
#define the IndianKanoon API class
class IndianKanoon:
  """
    https://github.com/aishik-pyne/kanoon
    Search query	https://api.indiankanoon.org/search/?formInput=<query>&pagenum=<pagenum>
    Document	https://api.indiankanoon.org/doc/<docid>/
    Document fragments	https://api.indiankanoon.org/docfragment/<docid>/?formInput=<query>
    Document Metainfo	https://api.indiankanoon.org/docmeta/<docid>/
  """

  def __init__(self):
    self.base_url = "https://api.indiankanoon.org/"
    #self.auth_token is a newly generated token from the IndiaKanoon API website(you have to generate it manually)
    self.auth_token = "86943a76a33aae7e711afee38d76ec0ea825476f"
    
    self.headers = {
        'authorization': "Token {}".format(self.auth_token),
        'cache-control': "no-cache",
    }
    self.api_session = requests.Session()
    self.api_session.headers = self.headers

  def search(self, formInput, pagenum=0,
             fromdate=None, todate=None,
             title=None, author=None,
             cite=None, bench=None, maxpages=50):
    #  Creating parameters
    params = {
        'formInput': formInput,
        'pagenum': pagenum,
        'maxpages': maxpages
    }
    if fromdate:
      assert isinstance(fromdate, datetime) 
      params['fromdate'] = fromdate.strftime('%d-%m-&Y')

    if todate:
      assert isinstance(todate, datetime) 
      params['todate'] = todate.strftime('%d-%m-&Y')

    # Making the request
    response = self.api_session.post(
        urljoin(self.base_url, 'search/'), params=params)
    response.raise_for_status()
    return response.json()

  def doc(self, docid):
    response = self.api_session.post(
        urljoin(self.base_url, 'doc/{}/'.format(docid)))
    response.raise_for_status()
    return response.json()

  def docfragment(self, docid, formInput):
    params = {
        'formInput': formInput,
    }
    response = self.api_session.post(
        urljoin(self.base_url, 'docfragment/{}/'.format(docid)), params=params)
    response.raise_for_status()
    return response.json()

  def docmeta(self, docid):
    response = self.api_session.post(
        urljoin(self.base_url, 'docmeta/{}/'.format(docid)))
    response.raise_for_status()
    return response.json()

In [45]:
#initialize object
ik = IndianKanoon()

In [5]:
#returns document id from link
def get_id(link):
    regex = re.compile(r'[0-9]+')
    result = regex.search(link)
    
    if(result):
        return result.group()
    else:
        return ""

In [7]:
oe_old = pd.read_csv("OE_LABELS.csv")

In [9]:
#get tids from file
res = []
for i in range(oe_old.shape[0]):
    res.append(int(oe_old.iloc[i,0]))

In [10]:
res

[928322,
 699319,
 1988649,
 1362442,
 357132,
 1722050,
 985842,
 617044,
 1885635,
 104810373,
 4354,
 174974,
 1341112,
 64332,
 334293,
 137050,
 30319311,
 832781,
 1567371,
 76636976,
 1168060,
 634266,
 1664373,
 42294818,
 1350547,
 199977,
 48216667,
 611025,
 762747,
 268937,
 104312764,
 353751,
 584252,
 1327080,
 1426304,
 1951505,
 91635,
 8919089,
 844819,
 634059,
 1013766,
 1187885,
 68469,
 1771435,
 557368,
 743158,
 1928398,
 309271,
 1295746,
 938898,
 1134697,
 1505859,
 310087,
 91986422,
 380655,
 1830927,
 33136745,
 912526,
 292286,
 1716282,
 429924,
 176039345,
 508915,
 810279,
 88087196,
 309307,
 69624144,
 170697,
 1154981,
 1474119,
 42355707,
 146940961,
 1278222,
 62991042,
 223504,
 31239895,
 582634,
 2671204,
 1030976,
 1905157,
 672966,
 528391,
 94788795,
 292442,
 52129591,
 1187886,
 1022584,
 486081,
 860600,
 444303,
 1341716,
 1877462,
 1500505,
 1353689,
 994554,
 6544121,
 6137338,
 1103984,
 465681,
 867423,
 1991497,
 1960330,
 109874394

In [41]:
res[80:]

[672966,
 528391,
 94788795,
 292442,
 52129591,
 1187886,
 1022584,
 486081,
 860600,
 444303,
 1341716,
 1877462,
 1500505,
 1353689,
 994554,
 6544121,
 6137338,
 1103984,
 465681,
 867423,
 1991497,
 1960330,
 109874394,
 71315,
 280485,
 1129833,
 639803,
 206804,
 36937,
 382721,
 379222,
 875590,
 1246653,
 590022,
 837614,
 176508424,
 39169380,
 37963,
 1641452,
 108020655,
 1700195,
 1697217,
 1831036,
 794955,
 33194727,
 962370,
 1271015,
 1615016,
 1053542,
 20356733,
 1304475,
 849254,
 241970,
 468834,
 1023674,
 1482259,
 653417,
 329158,
 199908142,
 1484358,
 403501,
 1147125,
 17789938,
 491934,
 1763592,
 296025,
 54995969,
 458739,
 1752565,
 96002,
 128161,
 1341895,
 165621647,
 980877,
 1725672,
 1130742,
 111777378,
 279427,
 1754074,
 784749,
 732612,
 1414629,
 857323,
 47808634,
 1867924,
 1333219,
 1186368,
 102886851,
 1610804,
 736105,
 1566,
 605322,
 1902038,
 74910860,
 51689837,
 6547,
 1844910,
 1024124,
 79561,
 119316583,
 1890994,
 982330,
 175238

In [46]:
docs = []

count = 0
for i in res[80:]:
    docs.append(ik.doc(docid=str(i)))
    print("FILE NUMBER: ",(count+1))
    count = count + 1

FILE NUMBER:  1
FILE NUMBER:  2
FILE NUMBER:  3
FILE NUMBER:  4
FILE NUMBER:  5
FILE NUMBER:  6
FILE NUMBER:  7
FILE NUMBER:  8
FILE NUMBER:  9
FILE NUMBER:  10
FILE NUMBER:  11
FILE NUMBER:  12
FILE NUMBER:  13
FILE NUMBER:  14
FILE NUMBER:  15
FILE NUMBER:  16
FILE NUMBER:  17
FILE NUMBER:  18
FILE NUMBER:  19
FILE NUMBER:  20
FILE NUMBER:  21
FILE NUMBER:  22
FILE NUMBER:  23
FILE NUMBER:  24
FILE NUMBER:  25
FILE NUMBER:  26
FILE NUMBER:  27
FILE NUMBER:  28
FILE NUMBER:  29
FILE NUMBER:  30
FILE NUMBER:  31
FILE NUMBER:  32
FILE NUMBER:  33
FILE NUMBER:  34
FILE NUMBER:  35
FILE NUMBER:  36
FILE NUMBER:  37
FILE NUMBER:  38
FILE NUMBER:  39
FILE NUMBER:  40
FILE NUMBER:  41
FILE NUMBER:  42
FILE NUMBER:  43
FILE NUMBER:  44
FILE NUMBER:  45
FILE NUMBER:  46
FILE NUMBER:  47
FILE NUMBER:  48
FILE NUMBER:  49
FILE NUMBER:  50
FILE NUMBER:  51
FILE NUMBER:  52
FILE NUMBER:  53
FILE NUMBER:  54
FILE NUMBER:  55
FILE NUMBER:  56
FILE NUMBER:  57
FILE NUMBER:  58
FILE NUMBER:  59
FILE N

FILE NUMBER:  463
FILE NUMBER:  464
FILE NUMBER:  465
FILE NUMBER:  466
FILE NUMBER:  467
FILE NUMBER:  468
FILE NUMBER:  469
FILE NUMBER:  470
FILE NUMBER:  471
FILE NUMBER:  472
FILE NUMBER:  473
FILE NUMBER:  474
FILE NUMBER:  475
FILE NUMBER:  476
FILE NUMBER:  477
FILE NUMBER:  478
FILE NUMBER:  479
FILE NUMBER:  480
FILE NUMBER:  481
FILE NUMBER:  482
FILE NUMBER:  483
FILE NUMBER:  484
FILE NUMBER:  485
FILE NUMBER:  486
FILE NUMBER:  487
FILE NUMBER:  488
FILE NUMBER:  489
FILE NUMBER:  490
FILE NUMBER:  491
FILE NUMBER:  492
FILE NUMBER:  493
FILE NUMBER:  494
FILE NUMBER:  495
FILE NUMBER:  496
FILE NUMBER:  497
FILE NUMBER:  498
FILE NUMBER:  499
FILE NUMBER:  500
FILE NUMBER:  501
FILE NUMBER:  502
FILE NUMBER:  503
FILE NUMBER:  504
FILE NUMBER:  505
FILE NUMBER:  506
FILE NUMBER:  507
FILE NUMBER:  508
FILE NUMBER:  509
FILE NUMBER:  510
FILE NUMBER:  511
FILE NUMBER:  512
FILE NUMBER:  513
FILE NUMBER:  514
FILE NUMBER:  515
FILE NUMBER:  516
FILE NUMBER:  517
FILE NUMBE

In [47]:
X = pd.DataFrame.from_dict(docs)

In [48]:
X

,tid,covertids,publishdate,firstname,secondname,lastname,title,filename,doc,numcites,...,covertitles,url,showurl,docsource,covers,citeList,citedbyList,divtype,relatedqs,courtcopy
0,672966,[],2001-10-05,s,None,aggarwal,Indian Herbs Research & Supply Co. ... vs Lalj...,www.manupatra.co.in/NXT/gateway.dll/highcourt1...,"<div class=""docsource_main"">Delhi High Court</...",21,...,[],None,True,Delhi High Court,[],"[{'tid': 762169, 'title': 'Section 62(2) in th...","[{'tid': 1031000, 'title': 'Rati Traders vs K....",judgments,"[{'value': 'passing off', 'formInput': 'passin...",False
1,528391,[],2011-03-25,None,None,None,M/S.Bade Miya vs Mubin Ahmed Zahurislam on 25 ...,bombay/2011-03-25/NMS-386-2011,"<div class=""docsource_main"">Bombay High Court<...",10,...,[],http://bombayhighcourt.nic.in/generatenew.php?...,True,Bombay High Court,[],"[{'tid': 1778816, 'title': 'Section 69 in The ...","[{'tid': 168001022, 'title': 'Aristo Pharmaceu...",judgments,"[{'value': 'trademark', 'formInput': 'trademar...",True
2,94788795,[],2017-09-04,None,None,None,Binoy Viswam vs Union Of India And Ors. on 4 S...,scorders/2017-09-04/binoy_viswam_union_of_indi...,"<div class=""docsource_main"">Supreme Court - Da...",0,...,[],http://www.sci.gov.in/supremecourt/2016/38664/...,True,Supreme Court - Daily Orders,[],[],"[{'tid': 101240501, 'title': 'Arun Jain (Huf) ...",judgments,NaN,True
3,292442,[],1971-02-19,k,None,roy,Tarulata Shyam And Ors. vs Commissioner Of Inc...,www.manupatra.co.in/NXT/gateway.dll/HighCourt2...,"<div class=""docsource_main"">Calcutta High Cour...",13,...,[],None,True,Calcutta High Court,[],"[{'tid': 545792, 'title': 'Section 2 in The In...","[{'tid': 109793757, 'title': 'The Deputy Commi...",judgments,"[{'value': 'section 2(e)', 'formInput': 'secti...",False
4,52129591,[],2020-03-17,None,None,None,Prof. Dr. Claudio De Simone & Anr. vs Actial F...,delhi/2020-03-17/CS(OS)-576-2019,"<div class=""docsource_main"">Delhi High Court</...",22,...,[],http://164.100.69.66/jupload/dhc/RSE/judgement...,True,Delhi High Court,[],"[{'tid': 1937976, 'title': 'The Patents Act, 1...",[],judgments,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,12050516,[],2019-05-30,None,None,None,"Terex India Pvt. Ltd., Tamil Nadu vs Dcit, New...",itat_delhi/2019-05-30/ITA-4791-DEL-2015,"<div class=""docsource_main"">Income Tax Appella...",13,...,[],https://www.itat.gov.in/files/uploads/category...,True,Income Tax Appellate Tribunal - Delhi,[],"[{'tid': 157492803, 'title': 'Bg Exploration &...",[],judgments,NaN,True
874,123737198,[],2016-01-20,b,d,ahmed,Ultra Home Construction Pvt. Ltd vs Purushotta...,delhi/2016-01-20/FAO(OS)--494-2015,"<div class=""docsource_main"">Delhi High Court</...",12,...,[],http://lobis.nic.in/ddir/dhc/BDA/judgement/20-...,True,Delhi High Court,[],"[{'tid': 1136195, 'title': 'the Copyright Act,...","[{'tid': 52383158, 'title': 'M/S Allied Blende...",judgments,"[{'value': ' FAO(OS)', 'formInput': '%20FAO%28...",True
875,1307808,[],2001-10-31,s,None,hegde,Union Of India & Ors vs Adani Exports Ltd. & A...,judis.nic.in/2001-10-31/UNION OF INDIA & ORS. ...,"<div class=""docsource_main"">Supreme Court of I...",6,...,[],http://judis.nic.in/supremecourt/imgst.aspx?fi...,True,Supreme Court of India,[],"[{'tid': 777058, 'title': 'Navinchandra N. Maj...","[{'tid': 33606507, 'title': 'Karthi P Chidamba...",judgments,"[{'value': 'territorial jurisdiction', 'formIn...",True
876,1747781,[],1996-07-09,g,None,ray,Vishnudas Trading As Vishnudas vs The Vazir Su...,judis.nic.in/1996-07-09/VISHNUDAS TRADING AS V...,"<div class=""docsource_main"">Supreme Court of I...",30,...,[],http://judis.nic.in/supremecourt/imgst.aspx?fi...,True,Supreme Court of India,[],"[{'tid': 1017213, 'title': 'The Trade Marks Ac...","[{'tid': 121128160, 'title': 'Mr.A.D.Padmasing...",judgments,"[{'value': 'tobacco', 'formInput': 'tobacco'},...",True


In [49]:
X.to_csv("81toLAST.csv",index=False)

In [50]:
df_with_tid_doc = pd.DataFrame(columns = ["DOC_ID","CONTENT"])
df_with_tid_doc["DOC_ID"] = X["tid"]
df_with_tid_doc["CONTENT"] = X["doc"]

In [51]:
h = html2text.HTML2Text()
h.ignore_links = True

In [52]:
#save html files

data = pd.read_csv("81toLAST.csv")

for i in range(data.shape[0]):
    f = open("html_files/"+str(int(data.iloc[i,0]))+".html","w+",encoding="utf-8")
    f.write(data.iloc[i,8])